In [18]:
import re

def get_data(file):
    f = open(file, "r")
    lines = [line[:-1] for line in f.readlines()]

    m = max([len(line) for line in lines[:-2]])
    A = [[] for i in range(len(lines[:-2]))]
    for i, line in enumerate(lines[:-2]):
        A[i] = line
        mi = len(line)
        A[i] += ' '*(m-mi)
    
    traj = lines[-1]
    lengths = [int(s) for s in re.findall(r'\d+', traj)]
    rotations = re.findall(r'[LR]+', traj)
    d = {
        'L': {'N': 'W', 'W': 'S', 'S': 'E', 'E': 'N'},
        'R': {'N': 'E', 'E': 'S', 'S': 'W', 'W': 'N'}
        }
    directions = []
    current_dir = 'E'
    directions.append(current_dir)
    for r in rotations:
        current_dir = d[r][current_dir]
        directions.append(current_dir)

    N_cols = len(A[0])
    ranges_byrow = []
    ranges_bycol = [[] for i in range(N_cols)]

    for i, line in enumerate(A):
        ranges_byrow.append([j for j, s in enumerate(line) if s != " "])
        for j in range(N_cols):
            if A[i][j]!=" ":
                ranges_bycol[j].append(i)
    return A, lengths, rotations, directions, ranges_byrow, ranges_bycol

file = "data/data22test.txt"
A, lengths, rotations, directions, ranges_byrow, ranges_bycol = get_data(file)
N_rows = len(A)
N_cols = len(A[0])
for line in A:
    print(len(line), line)


16         ...#    
16         .#..    
16         #...    
16         ....    
16 ...#.......#    
16 ........#...    
16 ..#....#....    
16 ..........#.    
16         ...#....
16         .....#..
16         .#......
16         ......#.


In [22]:
i = 0
j = A[0].index('.')
pos = (i,j)
print(pos)

for t in range(len(lengths)):

    l = lengths[t]
    dir = directions[t]
    if dir=='N':
        shift = (-1, 0)
    elif dir=='S':
        shift = (1, 0)
    if dir=='E':
        shift = (0, 1)
    elif dir=='W':
        shift = (0, -1)

    while l>0:
        nextpos = pos[0]+shift[0], pos[1]+shift[1]
        
        # check for wrapups
        if dir=='N': # go up
            if nextpos[0] not in ranges_bycol[nextpos[1]]: #nothing above, go on the bottom one
                nextpos = ranges_bycol[nextpos[1]][-1], nextpos[1]
        elif dir=='S': # go down
            if nextpos[0] not in ranges_bycol[nextpos[1]]: #nothing below, go on the top one
                nextpos = ranges_bycol[nextpos[1]][0], nextpos[1]
        elif dir=='W': # go left
            if nextpos[1] not in ranges_byrow[nextpos[0]]: #nothing on the left, go rightmost
                nextpos = nextpos[0], ranges_byrow[nextpos[0]][-1]
        elif dir=='E': # go right
            if nextpos[1] not in ranges_byrow[nextpos[0]]: #nothing one the right, go leftmost
                nextpos = nextpos[0], ranges_byrow[nextpos[0]][0]
        
        # check for blocking block
        nextchar = A[nextpos[0]][nextpos[1]]
        if nextchar=='#':
            l = 0
        elif nextchar=='.':
            pos = nextpos
            l -= 1
        else:
            print("euh... il y a un problème ici", nextpos)
            l -= 1
    
    print(pos, dir)

dir_score = {'E': 0, 'S': 1, 'W': 2, 'N': 3}

1000*(pos[0]+1) + 4*(pos[1]+1) + dir_score[dir]

(0, 8)
(0, 10) E
(5, 10) S
(5, 3) E
(7, 3) S
(7, 7) E
(5, 7) S
(5, 7) E


6032

In [23]:
# part 2
file = "data/data22test.txt"
A, lengths, rotations, directions, ranges_byrow, ranges_bycol = get_data(file)
N_rows = len(A)
N_cols = len(A[0])

print(lengths, rotations, directions, '\n')

print("   0123456789012345")
for i, line in enumerate(A):
    print(str(i).zfill(2), line)

[10, 5, 5, 10, 4, 5, 5] ['R', 'L', 'R', 'L', 'R', 'L'] ['E', 'S', 'E', 'S', 'E', 'S', 'E'] 

   0123456789012345
00         ...#    
01         .#..    
02         #...    
03         ....    
04 ...#.......#    
05 ........#...    
06 ..#....#....    
07 ..........#.    
08         ...#....
09         .....#..
10         .#......
11         ......#.


In [27]:
dict_next = {}
dir_next = {}

j = -1
for i in range(4, 8): # dir was W
    dict_next[(i, j)] = (11, 19-i)
    dir_next[(i, j)] = 'N'

j = 7
for i in range(0, 4): # dir was W
    dict_next[(i, j)] = (4, 4+i)
    dir_next[(i, j)] = 'S'
for i in range(8, 12): # dir was W
    dict_next[(i, j)] = (7, 15-i)
    dir_next[(i, j)] = 'N'

j = 12
for i in range(0, 4): # dir was E
    dict_next[(i, j)] = (11-i, 15)
    dir_next[(i, j)] = 'W'
for i in range(4, 8): # dir was E
    dict_next[(i, j)] = (8, 19-i)
    dir_next[(i, j)] = 'S'

j = 16
for i in range(8, 12): # dir was E
    dict_next[(i, j)] = (11-i, 11)
    dir_next[(i, j)] = 'W'

i = -1
for j in range(8, 12): # dir was N
    dict_next[(i, j)] = (4, 11-j)
    dir_next[(i, j)] = 'S'
i = 3
for j in range(4): # dir was N
    dict_next[(i, j)] = (0, 7-j)
    dir_next[(i, j)] = 'S'
for j in range(4, 8): # dir was N
    dict_next[(i, j)] = (j-4, 8)
    dir_next[(i, j)] = 'E'
i = 7
for j in range(12, 16): # dir was N
    dict_next[(i, j)] = (19-j, 12)
    dir_next[(i, j)] = 'W'
i = 8
for j in range(4): # dir was S
    dict_next[(i, j)] = (11, 11-j)
    dir_next[(i, j)] = 'N'
for j in range(4, 8): # dir was S
    dict_next[(i, j)] = (15-j, 8)
    dir_next[(i, j)] = 'E'
i = 12
for j in range(8, 12): # dir was S
    dict_next[(i, j)] = (7, 11-j)
    dir_next[(i, j)] = 'N'
for j in range(12, 16): # dir was S
    dict_next[(i, j)] = (19-j, 0)
    dir_next[(i, j)] = 'W'

d = {
    'L': {'N': 'W', 'W': 'S', 'S': 'E', 'E': 'N'},
    'R': {'N': 'E', 'E': 'S', 'S': 'W', 'W': 'N'}
    }

In [30]:
i = 0
j = A[0].index('.')
pos = (i,j)
print(pos)

for t in range(len(lengths)):

    l = lengths[t]
    if t==0:
        dir = 'E'
    else:
        dir = d[rotations[t-1]][dir]
    print("instruction:", l, 'steps in the', dir, 'direction')
    
    while l>0:
        if dir=='N':
            shift = (-1, 0)
        elif dir=='S':
            shift = (1, 0)
        if dir=='E':
            shift = (0, 1)
        elif dir=='W':
            shift = (0, -1)

        nextpos = pos[0]+shift[0], pos[1]+shift[1]
        nextdir = dir
        
        # check for wrapups
        if nextpos in dict_next.keys():
            print("direction switch from", dir, "to", dir_next[nextpos])
            nextdir = dir_next[nextpos]
            print("position switch from", nextpos, "to", dict_next[nextpos])
            nextpos = dict_next[nextpos]
            
        # check for blocking block
        nextchar = A[nextpos[0]][nextpos[1]]
        if nextchar=='#':
            l = 0
        elif nextchar=='.':
            pos = nextpos
            dir = nextdir
            l -= 1
        else:
            print("euh... il y a un problème ici", nextpos)
            l -= 1
    
    print("finished with", pos, dir)

dir_score = {'E': 0, 'S': 1, 'W': 2, 'N': 3}

1000*(pos[0]+1) + 4*(pos[1]+1) + dir_score[dir]

(0, 8)
instruction: 10 steps in the E direction
finished with (0, 10) E
instruction: 5 steps in the S direction
finished with (5, 10) S
instruction: 5 steps in the E direction
direction switch from E to S
position switch from (5, 12) to (8, 14)
finished with (10, 14) S
instruction: 10 steps in the W direction
finished with (10, 10) W
instruction: 4 steps in the S direction
direction switch from S to N
position switch from (12, 10) to (7, 1)
finished with (5, 1) N
instruction: 5 steps in the E direction
finished with (5, 6) E
instruction: 5 steps in the N direction
direction switch from N to E
position switch from (3, 6) to (2, 8)
finished with (4, 6) N


5031

In [31]:
# fold the cube for the large dataset
dict_next = {}
dir_next = {}

j = -1
for i in range(100, 150): # dir was W
    dict_next[(i, j)] = (49-(i-100), 50)
    dir_next[(i, j)] = 'E'
for i in range(150, 200): # dir was W
    dict_next[(i, j)] = (0, 50+(i-150))
    dir_next[(i, j)] = 'S'

j = 49
for i in range(0, 50): # dir was W
    dict_next[(i, j)] = (149-i, 0)
    dir_next[(i, j)] = 'E'
for i in range(50, 100): # dir was W
    dict_next[(i, j)] = (100, (i-50))
    dir_next[(i, j)] = 'S'

j = 50
for i in range(150, 200): # dir was E
    dict_next[(i, j)] = (149, 50+(i-150))
    dir_next[(i, j)] = 'N'

j = 100
for i in range(50, 100): # dir was E
    dict_next[(i, j)] = (49, 100+(i-50))
    dir_next[(i, j)] = 'N'
for i in range(100, 150): # dir was E
    dict_next[(i, j)] = (49-(i-100), 149)
    dir_next[(i, j)] = 'W'

j = 150
for i in range(0, 50): # dir was E
    dict_next[(i, j)] = (149-i, 99)
    dir_next[(i, j)] = 'W'

i = -1
for j in range(50, 100): # dir was N
    dict_next[(i, j)] = (150+(j-50), 0)
    dir_next[(i, j)] = 'E'
for j in range(100, 150): # dir was N
    dict_next[(i, j)] = (199, 0+(j-100))
    dir_next[(i, j)] = 'N'

i = 50
for j in range(100, 150): # dir was S
    dict_next[(i, j)] = (50+(j-100), 99)
    dir_next[(i, j)] = 'W'

i = 99
for j in range(0, 50): # dir was N
    dict_next[(i, j)] = (50+j, 50)
    dir_next[(i, j)] = 'E'

i = 150
for j in range(50, 100): # dir was S
    dict_next[(i, j)] = (150+(j-50), 49)
    dir_next[(i, j)] = 'W'

i = 200
for j in range(0, 50): # dir was S
    dict_next[(i, j)] = (0, 100+j)
    dir_next[(i, j)] = 'S'

d = {
    'L': {'N': 'W', 'W': 'S', 'S': 'E', 'E': 'N'},
    'R': {'N': 'E', 'E': 'S', 'S': 'W', 'W': 'N'}
    }

In [32]:
file = "data/data22.txt"
A, lengths, rotations, directions, ranges_byrow, ranges_bycol = get_data(file)
N_rows = len(A)
N_cols = len(A[0])

In [33]:
i = 0
j = A[0].index('.')
pos = (i,j)
print(pos)

for t in range(len(lengths)):

    l = lengths[t]
    if t==0:
        dir = 'E'
    else:
        dir = d[rotations[t-1]][dir]
    print(f"instruction {t}/{len(lengths)}: {l} steps in the {dir} direction")
    
    while l>0:
        if dir=='N':
            shift = (-1, 0)
        elif dir=='S':
            shift = (1, 0)
        if dir=='E':
            shift = (0, 1)
        elif dir=='W':
            shift = (0, -1)

        nextpos = pos[0]+shift[0], pos[1]+shift[1]
        nextdir = dir
        
        # check for wrapups
        if nextpos in dict_next.keys():
            #print("direction switch from", dir, "to", dir_next[nextpos])
            nextdir = dir_next[nextpos]
            #print("position switch from", nextpos, "to", dict_next[nextpos])
            nextpos = dict_next[nextpos]
            
        # check for blocking block
        nextchar = A[nextpos[0]][nextpos[1]]
        if nextchar=='#':
            l = 0
        elif nextchar=='.':
            pos = nextpos
            dir = nextdir
            l -= 1
        else:
            print("euh... il y a un problème ici", nextpos)
            l -= 1
    
    #print("finished with", pos, dir)

dir_score = {'E': 0, 'S': 1, 'W': 2, 'N': 3}

1000*(pos[0]+1) + 4*(pos[1]+1) + dir_score[dir]

(0, 50)
instruction 0/2001: 38 steps in the E direction
instruction 1/2001: 16 steps in the N direction
instruction 2/2001: 3 steps in the N direction
instruction 3/2001: 38 steps in the W direction
instruction 4/2001: 3 steps in the S direction
instruction 5/2001: 48 steps in the W direction
instruction 6/2001: 16 steps in the W direction
instruction 7/2001: 22 steps in the S direction
instruction 8/2001: 5 steps in the W direction
instruction 9/2001: 8 steps in the N direction
instruction 10/2001: 46 steps in the S direction
instruction 11/2001: 32 steps in the E direction
instruction 12/2001: 38 steps in the S direction
instruction 13/2001: 41 steps in the W direction
instruction 14/2001: 29 steps in the E direction
instruction 15/2001: 9 steps in the N direction
instruction 16/2001: 23 steps in the S direction
instruction 17/2001: 44 steps in the W direction
instruction 18/2001: 19 steps in the E direction
instruction 19/2001: 2 steps in the S direction
instruction 20/2001: 50 step

135297